In [1]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/ESAA_OB_HW/ESAA_Project/open/article_LDA9.csv')
df

,Unnamed: 0,articleID,Title,Content,Format,Language,userID,userCountry,userRegion,topic,GensimLDAtopic
0,0,ARTICLE_0000,19 Tips For Everyday Git Use,I've been using git full time for the past 4 y...,HTML,en,USER_0683,NaN,NaN,4,4
1,1,ARTICLE_0001,Intel buys computer vision startup Itseez to i...,Intel has acquired computer vision and machine...,HTML,en,USER_1129,NaN,NaN,9,3
2,2,ARTICLE_0002,Practical End-to-End Testing with Protractor,One of the reasons AngularJS is so great to wo...,HTML,en,USER_0256,NaN,NaN,4,4
3,3,ARTICLE_0003,Corporate venture growth in Brazil is another ...,Despite recent positive news and a renewed int...,HTML,en,USER_1304,NaN,NaN,9,3
4,4,ARTICLE_0004,Cross-channel user experiences with Drupal (aw...,"Last year around this time, I wrote that The B...",HTML,en,USER_0336,NaN,NaN,4,4
...,...,...,...,...,...,...,...,...,...,...,...
3003,3003,ARTICLE_3003,Como consumir conteúdo de qualidade em iOS - C...,"Quando iniciei minha jornada em Swift, saindo ...",HTML,pt,USER_0882,BR,MG,5,5
3004,3004,ARTICLE_3004,Aurelia 1.0 is Here!!!,It's been an amazing journey to get here and i...,HTML,en,USER_0220,NaN,NaN,4,4
3005,3005,ARTICLE_3005,Lessons from converting an app to 100% Kotlin ...,This is part one in a series of posts about Ko...,HTML,en,USER_1010,BR,SP,4,4
3006,3006,ARTICLE_3006,ITA está oferecendo 10 cursos gratuitos a dist...,"O Instituto Tecnológico de Aeronáutica (ITA) ,...",HTML,pt,USER_1210,NaN,NaN,5,5


In [ ]:
import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument

import time
import datetime

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
pd.set_option('display.max_colwidth', 200)
abst_texts = df["Content"]
abst_texts

0       I've been using git full time for the past 4 years, and I wanted to share the most practical tips that I've learned along the way. Hopefully, it will be useful to somebody out there. If you are co...
1       Intel has acquired computer vision and machine learning startup Itseez to develop better navigation for self-driving cars. The value of the deal was undisclosed. Founded in 2005 and based in San F...
2       One of the reasons AngularJS is so great to work with is that it was developed around the idea that testing is so important that it should be built into the framework. Every check-in of the Angula...
3       Despite recent positive news and a renewed interest of international investors in the country, with some even forecasting an approaching " tsumoney ," the crude reality is that Brazil is suffering...
4       Last year around this time, I wrote that The Big Reverse of Web would force a major re-architecture of the web to bring the right information, to the right pers

In [ ]:
# 단어 토큰화 수행
common_texts = [word_tokenize(abst_words.lower()) for abst_words in abst_texts if not abst_words in stopwords.words()]

In [ ]:
print(common_texts[0])

['i', "'ve", 'been', 'using', 'git', 'full', 'time', 'for', 'the', 'past', '4', 'years', ',', 'and', 'i', 'wanted', 'to', 'share', 'the', 'most', 'practical', 'tips', 'that', 'i', "'ve", 'learned', 'along', 'the', 'way', '.', 'hopefully', ',', 'it', 'will', 'be', 'useful', 'to', 'somebody', 'out', 'there', '.', 'if', 'you', 'are', 'completely', 'new', 'to', 'git', ',', 'i', 'suggest', 'reading', 'git', 'cheat', 'sheet', 'first', '.', 'this', 'article', 'is', 'aimed', 'at', 'somebody', 'who', 'has', 'been', 'using', 'git', 'for', 'three', 'months', 'or', 'more', '.', 'table', 'of', 'contexts', ':', 'three', 'stages', 'in', 'git', ',', 'and', 'how', 'to', 'move', 'between', 'them', 'git', 'reset', '--', 'hard', 'head', 'and', 'git', 'status', '-s', 'good', 'commit', 'messages', 'git', 'auto-completion', 'create', 'aliases', 'for', 'your', 'most', 'frequently', 'used', 'commands', '1.', 'parameters', 'for', 'better', 'logging', 'sample', 'command', 'git', 'log', '--', 'oneline', '--', 'gr

In [ ]:
# Tagged documents 는 doc2vec model의 input으로 사용

tagged_data = []

for i, doc in enumerate(common_texts):
    tagged = TaggedDocument(doc, [i])
    tagged_data.append(tagged)

tagged_data[0]

TaggedDocument(words=['i', "'ve", 'been', 'using', 'git', 'full', 'time', 'for', 'the', 'past', '4', 'years', ',', 'and', 'i', 'wanted', 'to', 'share', 'the', 'most', 'practical', 'tips', 'that', 'i', "'ve", 'learned', 'along', 'the', 'way', '.', 'hopefully', ',', 'it', 'will', 'be', 'useful', 'to', 'somebody', 'out', 'there', '.', 'if', 'you', 'are', 'completely', 'new', 'to', 'git', ',', 'i', 'suggest', 'reading', 'git', 'cheat', 'sheet', 'first', '.', 'this', 'article', 'is', 'aimed', 'at', 'somebody', 'who', 'has', 'been', 'using', 'git', 'for', 'three', 'months', 'or', 'more', '.', 'table', 'of', 'contexts', ':', 'three', 'stages', 'in', 'git', ',', 'and', 'how', 'to', 'move', 'between', 'them', 'git', 'reset', '--', 'hard', 'head', 'and', 'git', 'status', '-s', 'good', 'commit', 'messages', 'git', 'auto-completion', 'create', 'aliases', 'for', 'your', 'most', 'frequently', 'used', 'commands', '1.', 'parameters', 'for', 'better', 'logging', 'sample', 'command', 'git', 'log', '--',

In [ ]:
# 문서 벡터 사이즈 설정

vec_size = 100

# min_count (int, optional) : Ignores all words with total frequency lower than this.
#
# alpha : learning rate 초기값
# min_alpha : Learning rate will linearly drop to "min_alpha" as training progresses.
#
# dm=1이면 'Pharagraph Vector - Distributed Memory'(PV-DM)를 의미하고
# dm=0은 'Pharagraph Vector - Distributed Bag of Words'(PV-DBOW)를 의미
# Distributed memory(분산 메모리) 모델은 문서의 단어 순서를 보존하는 반면
# Distributed Bag of words는 단어 순서를 보존하지 않는 bag of words 접근 방식을 사용

model = doc2vec.Doc2Vec(vector_size=vec_size, alpha=0.025, min_alpha=0.00025, min_count=1, dm=1)

model.build_vocab(tagged_data)

print("◎ Tag Size(문서 크기): {}".format(len(model.dv)))

◎ Tag Size(문서 크기): 3008


In [ ]:
# model.corpus_total_words : 단어 수가 저장되어 있음

print("◎ 단어 수 = ", model.corpus_total_words)


◎ 단어 수 =  3084550


In [ ]:
# 모델 훈련
# 약 15초 정도 소요

max_epochs = 100
%time model.train(tagged_data, total_examples=model.corpus_count, epochs=max_epochs)

CPU times: user 16min 26s, sys: 7.86 s, total: 16min 34s
Wall time: 10min 41s


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

for epoch in range(10):
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    model.alpha -= 0.002
    model.min_alpha = model.alpha

# 각 문서의 벡터 추출
doc_vectors = [model.dv[i] for i in range(len(tagged_data))]

# 코사인 유사도 계산
cosine_sim_matrix = cosine_similarity(doc_vectors)

# 매트릭스 출력
print(cosine_sim_matrix)

KeyboardInterrupt: 

In [ ]:
# new sentence 의 벡터값 연산
# 벡터화할 문서는 리스트 형식이여야 함

new_sentence_vectorized = model.infer_vector(new_sentence)

In [ ]:
# cosine similarity가 가장 유사한 문건 10개를 추출

similar_sentences = model.docvecs.most_similar(positive=[new_sentence_vectorized], topn=10)


In [ ]:
# 유사한 문건 10개를 출력

similar_sentences


[(1359, 0.2597932815551758),
 (2290, 0.22594308853149414),
 (2507, 0.22128558158874512),
 (2430, 0.20690374076366425),
 (79, 0.20048090815544128),
 (135, 0.19797420501708984),
 (1399, 0.19668371975421906),
 (2839, 0.18995706737041473),
 (1508, 0.1885528862476349),
 (2350, 0.18807804584503174)]

In [ ]:
# 유사한 논문 10건을 판다스 형식으로 출력 확인

output = []

for i, v in enumerate(similar_sentences):
    index = v[0]
    output.append([index, df['Title'].loc[index], v[1]])

pd.DataFrame(output, columns=["index", "Title", "Similarity"])


,index,Title,Similarity
0,1359,Dilma regula Marco Civil e acaba com WhatsApp e Facebook grátis no celular,0.259793
1,2290,Drone believed to have hit British Airways flight 'may have been a plastic bag',0.225943
2,2507,[Tools] Universities should ban PowerPoint - It makes students stupid and professors boring,0.221286
3,2430,"DevOps security requires new mindset and tools for visibility, automation",0.206904
4,79,Drupal 8.1.0 is now available,0.200481
5,135,Good developers take breaks * Greetings Programs,0.197974
6,1399,A lição de um grande mestre de xadrez para resolver problemas,0.196684
7,2839,Beyond Progressive Web Apps Part 1,0.189957
8,1508,HotSpot 300,0.188553
9,2350,What Apple's differential privacy means for your data and the future of machine learning,0.188078


In [ ]:
subms = pd.read_csv('/content/drive/MyDrive/ESAA_OB_HW/ESAA_Project/open/sample_submission.csv')
subms

,userID,articleID
0,USER_0000,ARTICLE_0000
1,USER_0000,ARTICLE_0001
2,USER_0000,ARTICLE_0002
3,USER_0000,ARTICLE_0003
4,USER_0000,ARTICLE_0004
...,...,...
7070,USER_1420,ARTICLE_0000
7071,USER_1420,ARTICLE_0001
7072,USER_1420,ARTICLE_0002
7073,USER_1420,ARTICLE_0003


In [ ]:
# new sentence 설정


output = []
for i, val in enumerate(subms['articleID']):
    new_sentence = df.loc[df['articleID']== val].Content.values[0]
    new_sentence = new_sentence.split()
    new_sentence_vectorized = model.infer_vector(new_sentence)
    similar_sentences = model.docvecs.most_similar(positive=[new_sentence_vectorized], topn=5)
    for i, v in enumerate(similar_sentences):
        index = v[0]
        output.append([df['articleID'].loc[index], df['Title'].loc[index], v[1]])
pd.DataFrame(output, columns=["articleID", "Title", "Similarity"])

KeyboardInterrupt: 

In [ ]:
df

,articleID,Title,Content,Format,Language,userID,userCountry,userRegion
0,ARTICLE_0000,19 Tips For Everyday Git Use,"I've been using git full time for the past 4 years, and I wanted to share the most practical tips that I've learned along the way. Hopefully, it will be useful to somebody out there. If you are co...",HTML,en,USER_0683,NaN,NaN
1,ARTICLE_0001,Intel buys computer vision startup Itseez to improve navigation in self-driving cars,Intel has acquired computer vision and machine learning startup Itseez to develop better navigation for self-driving cars. The value of the deal was undisclosed. Founded in 2005 and based in San F...,HTML,en,USER_1129,NaN,NaN
2,ARTICLE_0002,Practical End-to-End Testing with Protractor,One of the reasons AngularJS is so great to work with is that it was developed around the idea that testing is so important that it should be built into the framework. Every check-in of the Angula...,HTML,en,USER_0256,NaN,NaN
3,ARTICLE_0003,Corporate venture growth in Brazil is another sign of a mature tech ecosystem,"Despite recent positive news and a renewed interest of international investors in the country, with some even forecasting an approaching "" tsumoney ,"" the crude reality is that Brazil is suffering...",HTML,en,USER_1304,NaN,NaN
4,ARTICLE_0004,Cross-channel user experiences with Drupal (awesome video demo!),"Last year around this time, I wrote that The Big Reverse of Web would force a major re-architecture of the web to bring the right information, to the right person, at the right time, in the right ...",HTML,en,USER_0336,NaN,NaN
...,...,...,...,...,...,...,...,...
3003,ARTICLE_3003,Como consumir conteúdo de qualidade em iOS - CocoaHeads BR,"Quando iniciei minha jornada em Swift, saindo da versão 1.1 para a 1.2, tive muuuuuitos problemas. Era a primeira vez que estava aprendendo uma linguagem em desenvolvimento (mas não sabia disso). ...",HTML,pt,USER_0882,BR,MG
3004,ARTICLE_3004,Aurelia 1.0 is Here!!!,"It's been an amazing journey to get here and it couldn't have happened without the feedback of thousands of community members, the contributions from hundreds of developers and the guidance of our...",HTML,en,USER_0220,NaN,NaN
3005,ARTICLE_3005,Lessons from converting an app to 100% Kotlin - Keepsafe Engineering,"This is part one in a series of posts about Kotlin. Click here for part 2, where I discuss the design of the Kotlin language . I've been following the development of Kotlin for a while. Kotlin is ...",HTML,en,USER_1010,BR,SP
3006,ARTICLE_3006,ITA está oferecendo 10 cursos gratuitos a distância - Engenharia é:,"O Instituto Tecnológico de Aeronáutica (ITA) , fundado em 1950, está oferecendo 10 cursos gratuitos por meio da plataforma de ensino on-line Coursera. A plataforma conta conta com mais de 12 milhõ...",HTML,pt,USER_1210,NaN,NaN


In [ ]:
df

,articleID,Title,Content,Format,Language,userID,userCountry,userRegion
0,ARTICLE_0000,19 Tips For Everyday Git Use,"I've been using git full time for the past 4 years, and I wanted to share the most practical tips that I've learned along the way. Hopefully, it will be useful to somebody out there. If you are co...",HTML,en,USER_0683,NaN,NaN
1,ARTICLE_0001,Intel buys computer vision startup Itseez to improve navigation in self-driving cars,Intel has acquired computer vision and machine learning startup Itseez to develop better navigation for self-driving cars. The value of the deal was undisclosed. Founded in 2005 and based in San F...,HTML,en,USER_1129,NaN,NaN
2,ARTICLE_0002,Practical End-to-End Testing with Protractor,One of the reasons AngularJS is so great to work with is that it was developed around the idea that testing is so important that it should be built into the framework. Every check-in of the Angula...,HTML,en,USER_0256,NaN,NaN
3,ARTICLE_0003,Corporate venture growth in Brazil is another sign of a mature tech ecosystem,"Despite recent positive news and a renewed interest of international investors in the country, with some even forecasting an approaching "" tsumoney ,"" the crude reality is that Brazil is suffering...",HTML,en,USER_1304,NaN,NaN
4,ARTICLE_0004,Cross-channel user experiences with Drupal (awesome video demo!),"Last year around this time, I wrote that The Big Reverse of Web would force a major re-architecture of the web to bring the right information, to the right person, at the right time, in the right ...",HTML,en,USER_0336,NaN,NaN
...,...,...,...,...,...,...,...,...
3003,ARTICLE_3003,Como consumir conteúdo de qualidade em iOS - CocoaHeads BR,"Quando iniciei minha jornada em Swift, saindo da versão 1.1 para a 1.2, tive muuuuuitos problemas. Era a primeira vez que estava aprendendo uma linguagem em desenvolvimento (mas não sabia disso). ...",HTML,pt,USER_0882,BR,MG
3004,ARTICLE_3004,Aurelia 1.0 is Here!!!,"It's been an amazing journey to get here and it couldn't have happened without the feedback of thousands of community members, the contributions from hundreds of developers and the guidance of our...",HTML,en,USER_0220,NaN,NaN
3005,ARTICLE_3005,Lessons from converting an app to 100% Kotlin - Keepsafe Engineering,"This is part one in a series of posts about Kotlin. Click here for part 2, where I discuss the design of the Kotlin language . I've been following the development of Kotlin for a while. Kotlin is ...",HTML,en,USER_1010,BR,SP
3006,ARTICLE_3006,ITA está oferecendo 10 cursos gratuitos a distância - Engenharia é:,"O Instituto Tecnológico de Aeronáutica (ITA) , fundado em 1950, está oferecendo 10 cursos gratuitos por meio da plataforma de ensino on-line Coursera. A plataforma conta conta com mais de 12 milhõ...",HTML,pt,USER_1210,NaN,NaN


In [ ]:
# new sentence 의 벡터값 연산
# 벡터화할 문서는 리스트 형식이여야 함

new_sentence_vectorized = model.infer_vector(new_sentence)


In [ ]:
# cosine similarity가 가장 유사한 문건 10개를 추출

similar_sentences = model.docvecs.most_similar(positive=[new_sentence_vectorized], topn=10)


In [ ]:
# 유사한 문건 10개를 출력

similar_sentences


[(15, 0.6302712559700012),
 (74, 0.5374502539634705),
 (66, 0.4228910803794861),
 (89, 0.4134577512741089),
 (49, 0.4099719524383545),
 (43, 0.37906613945961),
 (38, 0.37403997778892517),
 (52, 0.3594915568828583),
 (54, 0.35203245282173157),
 (131, 0.3505883812904358)]

In [ ]:
# 유사한 논문 10건을 판다스 형식으로 출력 확인

output = []

for i, v in enumerate(similar_sentences):
    index = v[0]
    output.append([index, df['title'].loc[index], v[1]])

pd.DataFrame(output, columns=["index", "Title", "Similarity"])


,index,Title,Similarity
0,15,통계학적 실험계획법을 이용한 복합 미생물이 포함된 Batch System에서 BTEX 제거 연구 : Removal of BTEX Compounds in Batch Systems with Different Mixed Cultures Using Statistical Design of Experiments,0.630271
1,74,통계학적 방법과 수치모델을 이용한 충청도 농촌지역 지하수내 질산성질소 특성 연구,0.537450
2,66,생태통계학의 서열화 방법 연구,0.422891
3,89,흙의 物理的 特性 試驗結果에 미치는 試驗誤差에 관한 統計學的 分析,0.413458
4,49,오염 수계에서의 미생물 군집에 영향을 미치는 환경요인들의 통계학적 분석,0.409972
5,43,고도하수처리장 모래여과시설의 통계학적 성능 평가에 관한 연구,0.379066
6,38,심박동수 변동성에 대한 통계학적 분석,0.374040
7,52,"서울시 기상인자와 미세먼지(PM10, PM2.5)의 통계학적 특성분석",0.359492
8,54,중엔트로피 합금의 구형 나노압입 거동에 대한 통계학적 분석,0.352032
9,131,인구통계학적 변인이 보육교사의 교육신념에 미치는 영향,0.350588


In [ ]:
df_l = pd.read_csv("/content/drive/MyDrive/WebTextMining/기말프로젝트/labelled_df.csv")
df_l = df_l.drop(	["Unnamed: 0.1","index","Unnamed: 0"],axis=1)
df_l.head()

,title,link,writer,assigned,year,grad,preAbstract,topic
0,지구통계학적 방식에 의한 물리탐사자료의 Down-Scaling 연구,https://www.riss.kr/search/detail/DetailView.d...,한성미,강원대학교 대학원,2011,국내석사,지구통계학은통계학의 특별한 한 분야로서 채취한 시료의 기지점 정보로부터 미지의 위체...,5
1,통계학적 분석법을 이용한 바이오디젤과 글리세롤 카보네이트의 효소적 동시 생산 최적화,https://www.riss.kr/search/detail/DetailView.d...,이영락,광운대학교 대학원,2015,국내박사,본 연구에서는 생촉매를 사용하여 얻어지는 친환경 에너지인 바이오디젤과 바이오디젤의 ...,7
2,남한 거주 북한이탈주민의 식생활행동에 관한 연구 : 인구통계학적 특성과 심리적 문화...,https://www.riss.kr/search/detail/DetailView.d...,이애란,이화여자대학교 대학원,2003,국내석사,본 연구는 인구통계학적인 특성과 심리적 문화적응전략에 따라 북한 이탈주민들의 사회 ...,4
3,노인의 인구 통계학적 변인에 따른 생활체육 참여와 생활 만족에 미치는 영향,https://www.riss.kr/search/detail/DetailView.d...,이영민,목포대학교 교육대학원,2007,국내석사,<Abstract>\n\n Purpose of this research enfor...,2
4,인구통계학적 속성이 공공기관 직원 개인평가에 미치는 영향 : A기관의 인사 및 다면...,https://www.riss.kr/search/detail/DetailView.d...,조대연,서울대학교 대학원,2019,국내석사,본 연구는 인구통계학적인 속성이 공공기관 개인평가 결과에 어떠한 영향을 미치는지를 ...,6
